### **DATA ANALYSIS FOR LAUNCHING A COMPETITIVE MOVIE STUDIO**

In [181]:
# import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

%matplotlib inline

## LOADING DATA

In [182]:
movies_df = pd.read_csv('Data/tmdb.movies.csv', index_col=0)
movies_df

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...,...,...
26512,"[27, 18]",488143,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1
26515,"[10751, 12, 28]",366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1


In [183]:
budgets_df = pd.read_csv('Data/tn.movie_budgets.csv')
budgets_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [184]:
gross_df = pd.read_csv('Data/bom.movie_gross.csv')
gross_df

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


In [185]:
info_df = pd.read_csv('Data/rt.movie_info.tsv',sep='\t')
info_df

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1555,1996,Forget terrorists or hijackers -- there's a ha...,R,Action and Adventure|Horror|Mystery and Suspense,NaN,NaN,"Aug 18, 2006","Jan 2, 2007",$,"33,886,034",106 minutes,New Line Cinema
1556,1997,The popular Saturday Night Live sketch was exp...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",NaN,NaN,88 minutes,Paramount Vantage
1557,1998,"Based on a novel by Richard Powell, when the l...",G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,NaN,"Jan 1, 1962","May 11, 2004",NaN,NaN,111 minutes,NaN
1558,1999,The Sandlot is a coming-of-age story about a g...,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans|Robert Gunter,"Apr 1, 1993","Jan 29, 2002",NaN,NaN,101 minutes,NaN


In [186]:
reviews_df = pd.read_csv('Data/rt.reviews.tsv',sep='\t', encoding='latin1')
reviews_df

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"
...,...,...,...,...,...,...,...,...
54427,2000,The real charm of this trifle is the deadpan c...,NaN,fresh,Laura Sinagra,1,Village Voice,"September 24, 2002"
54428,2000,NaN,1/5,rotten,Michael Szymanski,0,Zap2it.com,"September 21, 2005"
54429,2000,NaN,2/5,rotten,Emanuel Levy,0,EmanuelLevy.Com,"July 17, 2005"
54430,2000,NaN,2.5/5,rotten,Christopher Null,0,Filmcritic.com,"September 7, 2003"


In [187]:
info_df.head()


,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


## CLEANING DATA


In [188]:
#checking duplicates.
budgets_df.loc[:, "id"] = budgets_df["id"].astype(int)
print(budgets_df.duplicated().sum()) 

0


In [189]:
#checking for null values.
budgets_df.isnull().sum()


id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

In [190]:
movies_df.info(), budgets_df.info(), gross_df.info(), info_df.info(), reviews_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 26517 entries, 0 to 26516
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   genre_ids          26517 non-null  object 
 1   id                 26517 non-null  int64  
 2   original_language  26517 non-null  object 
 3   original_title     26517 non-null  object 
 4   popularity         26517 non-null  float64
 5   release_date       26517 non-null  object 
 6   title              26517 non-null  object 
 7   vote_average       26517 non-null  float64
 8   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 2.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie   

(None, None, None, None, None)

In [191]:
# Remove commas and convert columns to numbers
cols_to_convert = ['production_budget', 'domestic_gross', 'worldwide_gross']
for col in cols_to_convert:
    budgets_df[col] = budgets_df[col].replace('[\$,]', '', regex=True).astype(float)

# Check if conversion worked
budgets_df.dtypes


id                     int64
release_date          object
movie                 object
production_budget    float64
domestic_gross       float64
worldwide_gross      float64
dtype: object

In [192]:
gross_df['foreign_gross'] = pd.to_numeric(gross_df['foreign_gross'], errors='coerce')

# Check if conversion worked
gross_df.dtypes


title              object
studio             object
domestic_gross    float64
foreign_gross     float64
year                int64
dtype: object

In [193]:
movies_df.isnull().sum(), budgets_df.isnull().sum(), gross_df.isnull().sum(), info_df.isnull().sum(), reviews_df.isnull().sum()


(genre_ids            0
 id                   0
 original_language    0
 original_title       0
 popularity           0
 release_date         0
 title                0
 vote_average         0
 vote_count           0
 dtype: int64,
 id                   0
 release_date         0
 movie                0
 production_budget    0
 domestic_gross       0
 worldwide_gross      0
 dtype: int64,
 title                0
 studio               5
 domestic_gross      28
 foreign_gross     1355
 year                 0
 dtype: int64,
 id                 0
 synopsis          62
 rating             3
 genre              8
 director         199
 writer           449
 theater_date     359
 dvd_date         359
 currency        1220
 box_office      1220
 runtime           30
 studio          1066
 dtype: int64,
 id                0
 review         5563
 rating        13517
 fresh             0
 critic         2722
 top_critic        0
 publisher       309
 date              0
 dtype: int64)

In [194]:
gross_df['foreign_gross'] = gross_df['foreign_gross'].fillna(0)


In [195]:
info_df['director'] = info_df['director'].fillna('Unknown')
info_df['writer'] = info_df['writer'].fillna('Unknown')
info_df = info_df.drop(columns=['currency'])


In [196]:
reviews_df = reviews_df.dropna(subset=['review', 'rating'])  # Drop empty reviews & ratings
reviews_df['critic'] = reviews_df['critic'].fillna('Unknown')
reviews_df['publisher'] = reviews_df['publisher'].fillna('Unknown')


C:\Users\Administrator\AppData\Local\Temp\ipykernel_12276\976825917.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df['critic'] = reviews_df['critic'].fillna('Unknown')
C:\Users\Administrator\AppData\Local\Temp\ipykernel_12276\976825917.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df['publisher'] = reviews_df['publisher'].fillna('Unknown')


In [197]:
movies_df.isnull().sum(), gross_df.isnull().sum(), info_df.isnull().sum(), reviews_df.isnull().sum()


(genre_ids            0
 id                   0
 original_language    0
 original_title       0
 popularity           0
 release_date         0
 title                0
 vote_average         0
 vote_count           0
 dtype: int64,
 title              0
 studio             5
 domestic_gross    28
 foreign_gross      0
 year               0
 dtype: int64,
 id                 0
 synopsis          62
 rating             3
 genre              8
 director           0
 writer             0
 theater_date     359
 dvd_date         359
 box_office      1220
 runtime           30
 studio          1066
 dtype: int64,
 id            0
 review        0
 rating        0
 fresh         0
 critic        0
 top_critic    0
 publisher     0
 date          0
 dtype: int64)

In [198]:
gross_df['studio'] = gross_df['studio'].fillna('Unknown')
gross_df['domestic_gross'] = gross_df['domestic_gross'].fillna(0)


In [199]:
info_df['synopsis'] = info_df['synopsis'].fillna('No synopsis available')
info_df['rating'] = info_df['rating'].fillna('Not Rated')
info_df['genre'] = info_df['genre'].fillna('Unknown')
info_df['studio'] = info_df['studio'].fillna('Unknown')
info_df['runtime'] = info_df['runtime'].fillna(0)  # or use the mean runtime
info_df['box_office'] = info_df['box_office'].fillna(0)  # or keep NaN if uncertain


In [200]:
movies_df.isnull().sum(), gross_df.isnull().sum(), info_df.isnull().sum(), reviews_df.isnull().sum()


(genre_ids            0
 id                   0
 original_language    0
 original_title       0
 popularity           0
 release_date         0
 title                0
 vote_average         0
 vote_count           0
 dtype: int64,
 title             0
 studio            0
 domestic_gross    0
 foreign_gross     0
 year              0
 dtype: int64,
 id                0
 synopsis          0
 rating            0
 genre             0
 director          0
 writer            0
 theater_date    359
 dvd_date        359
 box_office        0
 runtime           0
 studio            0
 dtype: int64,
 id            0
 review        0
 rating        0
 fresh         0
 critic        0
 top_critic    0
 publisher     0
 date          0
 dtype: int64)

# MERGING.

In [ ]:
movies_gross_df.info()


In [ ]:
movies_final_df = movies_gross_df.merge(info_df, on='id', how='left')


In [ ]:
movies_final_df.info()


In [ ]:
movies_final_df = movies_final_df.merge(reviews_df, on='id', how='left')


In [ ]:
movies_final_df.info()
movies_final_df.head()


In [ ]:
movies_final_df = movies_gross_df.merge(info_df[['id', 'genre']], on='id', how='left')


In [ ]:
movies_final_df.info()
movies_final_df[['title_clean', 'genre']].head(10)


In [ ]:
print("Sample title_clean in movies_gross_df:")
print(movies_gross_df['title_clean'].head(10))

print("\nSample title_clean in budgets_df:")
print(budgets_df['title_clean'].head(10))


In [ ]:
movies_gross_df['title_clean'] = movies_gross_df['title_clean'].str.strip().str.lower()
budgets_df['title_clean'] = budgets_df['title_clean'].str.strip().str.lower()


In [ ]:
matched_titles = movies_gross_df[movies_gross_df['title_clean'].isin(budgets_df['title_clean'])]
print("Number of matching titles:", matched_titles.shape[0])


In [ ]:
movies_final_df = movies_gross_df.merge(
    budgets_df[['title_clean', 'production_budget', 'domestic_gross', 'worldwide_gross']], 
    on='title_clean', 
    how='left'
)


In [ ]:
print(budgets_df.columns)


In [ ]:
print(budgets_df[['title_clean', 'production_budget', 'domestic_gross', 'worldwide_gross']].head())


In [ ]:
print(movies_gross_df[['title_clean']].head())
print(budgets_df[['title_clean']].head())


In [ ]:
movies_gross_df['title_clean'] = movies_gross_df['title_clean'].str.strip().str.lower()
budgets_df['title_clean'] = budgets_df['title_clean'].str.strip().str.lower()


In [ ]:
budgets_df = budgets_df.rename(columns=lambda x: x.strip())  # Remove extra spaces
print(budgets_df.columns)


In [ ]:
movies_final_df = movies_gross_df.merge(
    budgets_df[['title_clean', 'production_budget', 'domestic_gross', 'worldwide_gross']], 
    on='title_clean', 
    how='left'
)


In [ ]:
print(budgets_df.columns)


In [ ]:
budgets_df = budgets_df.rename(columns=lambda x: x.strip())  # Remove spaces
print(budgets_df.columns)


In [ ]:
for col in budgets_df.columns:
    print(f"'{col}'")


In [ ]:
budgets_df.rename(columns={' domestic_gross': 'domestic_gross'}, inplace=True)


In [ ]:
movies_final_df = movies_gross_df.merge(
    budgets_df[['title_clean', 'production_budget', 'domestic_gross', 'worldwide_gross']], 
    on='title_clean', 
    how='left'
)


In [ ]:
print(movies_final_df.columns)


In [ ]:
print(movies_final_df[['domestic_gross_x', 'domestic_gross_y']].head(10))


In [ ]:
movies_final_df['domestic_gross'] = movies_final_df['domestic_gross_y'].fillna(movies_final_df['domestic_gross_x'])
movies_final_df.drop(columns=['domestic_gross_x', 'domestic_gross_y'], inplace=True)


In [ ]:
print(movies_final_df.columns)  # Should show 'domestic_gross'
print(movies_final_df[['title_clean', 'domestic_gross']].head(10))  # Check values


In [ ]:
missing_domestic = movies_final_df[movies_final_df['domestic_gross'].isna()]
print(missing_domestic[['title_clean']].head(20))  # See if titles look correct


In [ ]:
print(budgets_df[budgets_df['title_clean'].str.contains("harry potter", case=False, na=False)])


In [ ]:
movies_final_df['domestic_gross'].fillna(0, inplace=True)


In [ ]:
movies_final_df['profit'] = movies_final_df['worldwide_gross'] - movies_final_df['production_budget']
movies_final_df['roi'] = (movies_final_df['profit'] / movies_final_df['production_budget']) * 100


In [ ]:
movies_final_df[['title_clean', 'profit', 'roi']].sort_values(by='profit', ascending=False).head(10)


In [ ]:
movies_final_df = movies_final_df.dropna(subset=['genre_ids'])  # Remove NaN genres
movies_final_df['genre_ids'] = movies_final_df['genre_ids'].str.split(', ')  # Split genre strings
movies_exploded = movies_final_df.explode('genre_ids')  # Expand into separate rows


In [ ]:
genre_profit = movies_exploded.groupby('genre_ids')['profit'].mean().sort_values(ascending=False)
print(genre_profit.head(10))  # Top 10 profitable genres


In [ ]:
# Remove brackets and extra characters
movies_exploded['genre_ids'] = movies_exploded['genre_ids'].astype(str).str.replace(r'[\[\]]', '', regex=True)


In [ ]:
genre_profit = movies_exploded.groupby('genre_ids')['profit'].mean().sort_values(ascending=False)
print(genre_profit.head(10))


In [ ]:
import ast

def extract_first_genre(genre):
    """Extracts the first genre ID from a list or keeps it as is if already clean."""
    if isinstance(genre, str):  # If it's a string, try to convert it to a list
        try:
            genre_list = ast.literal_eval(genre)  # Convert string to list safely
            if isinstance(genre_list, list) and len(genre_list) > 0:
                return int(genre_list[0])  # Take the first genre
        except (ValueError, SyntaxError):
            return None  # If conversion fails, return None
    return genre  # If it's already an integer, keep it

# Apply the function to clean genre_ids
movies_exploded['genre_ids'] = movies_exploded['genre_ids'].apply(extract_first_genre)

# Drop any rows where genre_id is still NaN
movies_exploded = movies_exploded.dropna(subset=['genre_ids'])

# Convert to int
movies_exploded['genre_ids'] = movies_exploded['genre_ids'].astype(int)

# Map genre IDs to names
movies_exploded['genre_ids'] = movies_exploded['genre_ids'].map(genre_mapping)

# Check the cleaned data
print(movies_exploded[['title_clean', 'genre_ids']].head(10))


In [ ]:
merged_df.head()


In [ ]:
merged_df.columns


In [ ]:
merged_df.isnull().sum()


In [ ]:
merged_df.dtypes


In [ ]:
# Fill missing release dates
merged_df["release_date_y"].fillna(merged_df["release_date_x"], inplace=True)

# Fill missing movie titles
merged_df["movie_y"].fillna(merged_df["movie_x"], inplace=True)

# Fill missing budget and gross with available values
merged_df["production_budget_y"].fillna(merged_df["production_budget_x"], inplace=True)
merged_df["worldwide_gross_y"].fillna(merged_df["worldwide_gross_x"], inplace=True)

# Drop duplicate or unnecessary columns
merged_df = merged_df.drop(columns=["release_date_x", "movie_x", "production_budget_x", "worldwide_gross_x", "title_x"])

# Rename columns for clarity
merged_df.rename(columns={"release_date_y": "release_date", "movie_y": "movie", "production_budget_y": "production_budget", "worldwide_gross_y": "worldwide_gross", "title_y": "title"}, inplace=True)

# Check if missing values are reduced
print(merged_df.isnull().sum())


In [ ]:
# Drop duplicate or unnecessary ID column
merged_df.drop(columns=["id_y"], inplace=True)

# Fill missing movie titles with title_clean if available
merged_df["movie"].fillna(merged_df["title_clean"], inplace=True)
merged_df["title"].fillna(merged_df["title_clean"], inplace=True)

# Fill missing release dates
merged_df["release_date"].fillna(method='ffill', inplace=True)

# Fill missing financial data (if another source isn't available, consider dropping)
merged_df["production_budget"].fillna(0, inplace=True)
merged_df["worldwide_gross"].fillna(0, inplace=True)
merged_df["domestic_gross"].fillna(0, inplace=True)
merged_df["foreign_gross"].fillna(0, inplace=True)

# Drop rows where movie titles are still missing
merged_df.dropna(subset=["movie"], inplace=True)

# Check for remaining missing values
print(merged_df.isnull().sum())


In [ ]:
type(merged_df)


In [ ]:
print(merged_df.dtypes)


In [ ]:
cols_to_fix = ["production_budget", "worldwide_gross", "domestic_gross", "foreign_gross"]
for col in cols_to_fix:
    merged_df[col] = pd.to_numeric(merged_df[col], errors="coerce")


In [ ]:
print(merged_df.dtypes)


In [ ]:
merged_df.to_csv("cleaned_movies_data.csv", index=False)


In [ ]:
merged_df.info()

In [ ]:
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]


In [ ]:
merged_df[['production_budget', 'domestic_gross', 'worldwide_gross']] = merged_df[
    ['production_budget', 'domestic_gross', 'worldwide_gross']
].fillna(0)


In [ ]:
merged_df.drop(columns=['studio', 'year'], inplace=True)


In [ ]:
merged_df.info()

In [ ]:
merged_df['foreign_gross'].fillna(0, inplace=True)  # Replace NaN with 0


In [ ]:
merged_df.info()

In [202]:
merged_df.head()

,genre_ids,id_x,original_language,original_title,popularity,vote_average,vote_count,title_clean,release_date,movie,production_budget,worldwide_gross,title,domestic_gross,foreign_gross
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,7.7,10788,harry potter and the deathly hallows: part 1,2010-11-19,harry potter and the deathly hallows: part 1,0.0,0.0,harry potter and the deathly hallows: part 1,0.0,0.0
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,7.7,7610,how to train your dragon,2010-03-26,How to Train Your Dragon,0.0,0.0,How to Train Your Dragon,0.0,277300000.0
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,6.8,12368,iron man 2,2010-05-07,Iron Man 2,0.0,0.0,Iron Man 2,0.0,311500000.0
3,"[16, 35, 10751]",862,en,Toy Story,28.005,7.9,10174,toy story,1995-11-22,Toy Story,0.0,0.0,toy story,0.0,0.0
4,"[28, 878, 12]",27205,en,Inception,27.920,8.3,22186,inception,2010-07-16,Inception,0.0,0.0,Inception,0.0,535700000.0


In [ ]:
merged_df.to_csv("cleaned_movies_data.csv", index=False)
